# 1) Import library

In [1]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModel  # NOT NEEDED FOR THE APP
import ast


import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

/home/lyxor/miniconda3/envs/anime/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda device


# 2) Import files 

In [2]:
# profiles 
df_profiles = pd.read_csv("https://anime-recommendation-engine.s3.eu-west-3.amazonaws.com/data/profiles_clean.csv")

In [3]:
# synopsis embedding MiniLM
df_MiniLM = pd.read_json('https://anime-recommendation-engine.s3.eu-west-3.amazonaws.com/data/synopsis_embedding.json')

In [ ]:
# NOT NEEDED FOR THE APP :  synopsis embedding Jina_1024   
df_jina_1024= pd.read_json('https://anime-recommendation-engine.s3.eu-west-3.amazonaws.com/data/synopsis_embedding_jina_1024.json')

# 3) preprocess

In [4]:
def preprocess_user(df):
    df["favorites_anime"] = df["favorites_anime"].apply(ast.literal_eval)
    df_favorites = df[["profile", "favorites_anime"]].copy().explode("favorites_anime")
    df_favorites = df_favorites.dropna(subset=["favorites_anime"])
    df_favorites["favorites_anime"] = df_favorites["favorites_anime"].astype("int64")

    return df_favorites

In [5]:
df_favorites = preprocess_user(df_profiles)

# MiniLM

In [6]:
Model_MiniLM = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [7]:
# similarity >= 0.5  
def search_closest_by_content_simi_50(content, model, df_emb, df_favorites, filter):

        content = re.sub("[^A-Za-z]+", " ", str(content)).lower()
        content = model.encode(content)

        # cosine similarity : given embedding VS all embeddings
        similarities = cosine_similarity([content], list(df_emb[filter]))[0]

        # Store similarity
        similarity_df = pd.DataFrame({'uid': df_emb['uid'], 'similarity': similarities})

        # filter by similarity
        closest = similarity_df[similarity_df['similarity'] >= 0.5].sort_values(by='similarity', ascending=False)

        df_merged = df_favorites.merge(closest, left_on='favorites_anime', right_on='uid', how='inner')
        grouped_df = df_merged.groupby('profile').agg({'uid': list}).reset_index()

        return grouped_df


# Top5_similarity  # NOT NEEDED FOR THE APP : 
def search_closest_by_content_top_5(content, model, df_emb, df_favorites, filter):

        content = re.sub("[^A-Za-z]+", " ", str(content)).lower()
        content = model.encode(content)

        # cosine similarity : given embedding VS all embeddings
        similarities = cosine_similarity([content], list(df_emb[filter]))[0]

        # Store similarity
        similarity_df = pd.DataFrame({'uid': df_emb['uid'], 'similarity': similarities})

        # filter by similarity.
        closest = similarity_df.sort_values(by='similarity', ascending=False).head(5)


        df_merged = df_favorites.merge(closest, left_on='favorites_anime', right_on='uid', how='inner')
        grouped_df = df_merged.groupby('profile').agg({'uid': list}).reset_index()

        return grouped_df


In [10]:
new_content = """During their decade-long quest to defeat the Demon King, the members of the hero's party—Himmel himself, the priest Heiter, the dwarf warrior Eisen, and the elven mage Frieren—forge bonds through adventures and battles, creating unforgettable precious memories for most of them.

However, the time that Frieren spends with her comrades is equivalent to merely a fraction of her life, which has lasted over a thousand years. When the party disbands after their victory, Frieren casually returns to her "usual" routine of collecting spells across the continent. Due to her different sense of time, she seemingly holds no strong feelings toward the experiences she went through.

As the years pass, Frieren gradually realizes how her days in the hero's party truly impacted her. Witnessing the deaths of two of her former companions, Frieren begins to regret having taken their presence for granted; she vows to better understand humans and create real personal connections. Although the story of that once memorable journey has long ended, a new tale is about to begin."""


filter = 'synopsis_embedding'
df_emb = df_MiniLM
df_favorites = df_favorites
model = Model_MiniLM

# call the function and store in DF   # NOT NEEDED FOR THE APP : 
Top_5_df = pd.DataFrame(search_closest_by_content_top_5(new_content, model, df_emb, df_favorites, filter))
display(Top_5_df.shape[0])


# call the function and store in DF
simi_50_df = pd.DataFrame(search_closest_by_content_simi_50(new_content, model, df_emb, df_favorites, filter))
display(simi_50_df.shape[0])


47

11

 # jina emb = 1024  (abandonned)

In [10]:
# Initialize the model
Model_jina_1024 = AutoModel.from_pretrained("jinaai/jina-embeddings-v3", trust_remote_code=True).to('cuda')

/home/lyxor/miniconda3/envs/DeepLearn/lib/python3.10/site-packages/flash_attn/ops/triton/layer_norm.py:985: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/home/lyxor/miniconda3/envs/DeepLearn/lib/python3.10/site-packages/flash_attn/ops/triton/layer_norm.py:1044: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout, *args):


In [11]:
# Function to get embeddings
def get_embeddings(text):
    embeddings = model.encode([text], task="text-matching")
    return embeddings[0]

# Top5_similarity
def search_closest_by_content_jina(content, model, df_emb, df_favorites, filter):

        content = re.sub("[^A-Za-z]+", " ", str(content)).lower()
        content = get_embeddings(content)

        # cosine similarity : given embedding VS all embeddings
        similarities = cosine_similarity([content], list(df_emb[filter]))[0]

        # Store similarity
        similarity_df = pd.DataFrame({'uid': df_emb['uid'], 'similarity': similarities})

        # filter by similarity. given_uid exclude
        closest = similarity_df.sort_values(by='similarity', ascending=False).head(5)


        df_merged = df_favorites.merge(closest, left_on='favorites_anime', right_on='uid', how='inner')
        grouped_df = df_merged.groupby('profile').agg({'uid': list}).reset_index()

        return grouped_df

In [16]:
filter = 'synopsis_embedding'
df_emb = df_jina_1024
df_favorites = df_favorites
model = Model_jina_1024


# call the function and store in DF
jina_df = pd.DataFrame(search_closest_by_content_jina(new_content, model, df_emb, df_favorites, filter))
display(jina_df.shape[0])

393

### we will stay on the miniLM embedding, for calculation and standardise with other functions.  Similarity >= 0.5